### General idea
1. Clean dataset => dfClean
2. Vectorize words => to probability density
3. Perform logistic regression on vectorized words 
    of scales of reviews 0 (0,.1),1 (.2,.3) ,2 (.4,.5),3 (.6,.7) ,4 (.8,.9, 1) reviews

In [1]:
# Read data set and stop words
import pandas as pd
import re 
import nltk
nltk.download('stopwords')
nltk.download('words')
rawData = pd.read_csv (r'.\AppReview.csv')
#df=df1.sample(n=1000)
len(rawData.index)
rawData.columns



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danxg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\danxg\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Index(['appID', 'reviewerName', 'reviewText', 'reviewerRating', 'reviewDate',
       'textAnalytics'],
      dtype='object')

In [5]:
rawData_v1=rawData[rawData["reviewerName"]!="A Google User"]
rawData_v1["reviewerRating"]=rawData["reviewerRating"]*10/2
rawData_v1=rawData_v1.astype({"reviewerRating": int})
rawData_v1.head()

C:\Users\danxg\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,appID,reviewerName,reviewText,reviewerRating,reviewDate,textAnalytics
0,3,Eric Hansen,Love it! WELL worth the money for the full ver...,5,2017/07/07 00:00:00,NaN
1,3,Jacob N.,There's an awful bug that doesn't allow you to...,2,2017/08/29 00:00:00,NaN
2,3,Higgins Family,Would be 5 stars except for the bugs.... For e...,4,2017/10/02 00:00:00,NaN
3,3,Rajko Dikmann,Worked perfect until a few weeks ago. Then bro...,2,2017/09/28 00:00:00,NaN
4,3,Sergei Garcia,Hands down the best browser on the play store!...,5,2017/07/09 00:00:00,NaN


In [ ]:
rawData_v1.groupby("reviewerRating").nunique()

In [34]:
# frames = []
# for i in range(1,6,1):
#     frames.append(rawData_v1[rawData_v1["reviewerRating"]==i/10].sample(n=1000, replace=False))

# df=pd.concat(frames,ignore_index =True)
# len(df.index)
#df=rawData_v1.sample(n=5000)
df=rawData_v1
#df.head()
df.groupby("reviewerRating").size()

reviewerRating
1     8501
2     4376
3     7571
4    15218
5    55085
dtype: int64

In [35]:
#df.shape[0]

In [36]:
# Corpus of stop words
from nltk.corpus import stopwords
from nltk.corpus import words


In [37]:
# processes a review and returns a list of words
def review_to_words(review, string = True, remove_stopwords=True):
    # Remove HTML
    #review_text = BeautifulSoup(review).get_text()
    review_text=review
    # Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # Convert words to lower case and split them
    thesewords = review_text.lower().split()
    # Ignore non=english words
#     englishWords = words.words()
#     thesewords = [w for w in thesewords if w in englishWords]

    
    # Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        thesewords = [w for w in thesewords if not w in stops]
    if string:
        return " ".join(thesewords)
    else:
        return thesewords

In [38]:
#Clean up text
#Remove non-ascii text
#Remove all rows missing reviewerName
def fixString(x):
    return x.encode('ascii',errors='ignore')

# df[["reviewText"]]=df[["reviewText"]].apply(lambda x: str(x["reviewText"]).encode('ascii',errors='ignore').decode(), axis=1)
# df[["reviewerName"]]=df[["reviewerName"]].apply(lambda x: str(x["reviewerName"]).encode('ascii',errors='ignore').decode(), axis=1)
df[["reviewText"]]=df[["reviewText"]].apply(lambda x: review_to_words(x["reviewText"]), axis=1)
df[["reviewerName"]]=df[["reviewerName"]].apply(lambda x: str(x["reviewerName"]).encode('ascii',errors='ignore').decode(), axis=1)

dfCleaned=df[df['reviewText'].str.strip().astype(bool)]
dfCleaned=dfCleaned[df['reviewerName'].str.strip().astype(bool)]

#
#dfCleaned

In [39]:
storedDFCleaned="dfCleaned_raw"
dfCleaned.to_pickle(storedDFCleaned)
#dfCleaned=pd.read_pickle(storedDFCleaned)

## Vectorize words
### This is based on 
https://towardsdatascience.com/latent-semantic-analysis-sentiment-classification-with-python-5f657346f6a3


In [40]:
# import statements
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit(dfCleaned["reviewText"])

X = tfidf.transform(dfCleaned["reviewText"])

[0.0]


In [42]:
dfCleaned.iloc[1]["reviewText"]

'awful bug allow use space bar want type search term url box ruined whole experience free browsers like dee browser one without awful bugs believe paid app full review'

In [46]:
print([X[1, tfidf.vocabulary_['awful']]])

[0.4528846823706622]


### Modelling part
1. Leverage the raw vector count and the tf-idf weighted version

In [47]:
df=dfCleaned

In [48]:
import numpy as np
df[df['reviewerRating'] != 3]
df['Positivity'] = np.where(df['reviewerRating'] > 3, 1, 0)
cols = ['appID', 'reviewerName', 'reviewerRating', 'reviewDate', 'textAnalytics']
df.drop(cols, axis=1, inplace=True)
df.dropna(inplace=True)
df.head()

,reviewText,Positivity
0,love well worth money full version came ad blo...,1
1,awful bug allow use space bar want type search...,0
2,would stars except bugs example incognito tab ...,1
3,worked perfect weeks ago browsing experience s...,0
4,hands best browser play store even flagship de...,1


In [49]:
from sklearn.model_selection import train_test_split
X = df.reviewText
y = df.Positivity
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
print("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(X_train),
                                                                             (len(X_train[y_train == 0]) / (len(X_train)*1.))*100,
                                                                            (len(X_train[y_train == 1]) / (len(X_train)*1.))*100))

Train set has total 67566 entries with 22.63% negative, 77.37% positive


In [50]:
print("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(X_test),
                                                                             (len(X_test[y_test == 0]) / (len(X_test)*1.))*100,
                                                                            (len(X_test[y_test == 1]) / (len(X_test)*1.))*100))

Test set has total 22522 entries with 22.32% negative, 77.68% positive


In [51]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
def accuracy_summary(pipeline, X_train, y_train, X_test, y_test):
    sentiment_fit = pipeline.fit(X_train, y_train)
    y_pred = sentiment_fit.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("accuracy score: {0:.2f}%".format(accuracy*100))
    return accuracy

In [52]:
cv = CountVectorizer()
rf = RandomForestClassifier(class_weight="balanced")
n_features = np.arange(10000,30001,10000)
def nfeature_accuracy_checker(vectorizer=cv, n_features=n_features, stop_words=None, ngram_range=(1, 1), classifier=rf):
    result = []
    print(classifier)
    print("\n")
    for n in n_features:
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print("Test result for {} features".format(n))
        nfeature_accuracy = accuracy_summary(checker_pipeline, X_train, y_train, X_test, y_test)
        result.append((n,nfeature_accuracy))
    return result
tfidf = TfidfVectorizer()
print("Result for trigram with stop words (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(vectorizer=tfidf,ngram_range=(1, 3))

Result for trigram with stop words (Tfidf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


Test result for 10000 features
accuracy score: 85.80%
Test result for 20000 features
accuracy score: 86.06%
Test result for 30000 features
accuracy score: 86.06%


In [53]:
from sklearn.metrics import classification_report
cv = CountVectorizer(max_features=30000,ngram_range=(1, 3))
pipeline = Pipeline([
        ('vectorizer', cv),
        ('classifier', rf)
    ])
sentiment_fit = pipeline.fit(X_train, y_train)
y_pred = sentiment_fit.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['negative','positive']))

              precision    recall  f1-score   support

    negative       0.73      0.59      0.65      5026
    positive       0.89      0.94      0.91     17496

    accuracy                           0.86     22522
   macro avg       0.81      0.76      0.78     22522
weighted avg       0.85      0.86      0.85     22522



In [56]:
X_train

60163    great app org mode files wish free freedom sup...
84085    fan hi really keen activating super sensor one...
17149          liked mh u player must app haha full review
56477    ok miui seems ignore dnd setting would thought...
60868    errors extremely useful easy use often search ...
                               ...                        
25195    comes handy especially age social media nevert...
53324                                    works full review
50026          really works great really works full review
50996                                     nice full review
80700           favourite move material design full review
Name: reviewText, Length: 67566, dtype: object

In [57]:
#Logistic regression?
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter=1000, multi_class='auto')

cv = CountVectorizer()
n_features = np.arange(10000,30001,10000)
def nfeature_accuracy_checker(vectorizer=cv, n_features=n_features, stop_words=None, ngram_range=(1, 1), classifier=clf):
    result = []
    print(classifier)
    print("\n")
    for n in n_features:
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print("Test result for {} features".format(n))
        nfeature_accuracy = accuracy_summary(checker_pipeline, X_train, y_train, X_test, y_test)
        result.append((n,nfeature_accuracy))
    return result
tfidf = TfidfVectorizer()
print("Result for trigram with stop words (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(vectorizer=tfidf,ngram_range=(1, 3))

Result for trigram with stop words (Tfidf)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


Test result for 10000 features
accuracy score: 87.31%
Test result for 20000 features
accuracy score: 87.34%
Test result for 30000 features
accuracy score: 87.44%


In [59]:
from sklearn.metrics import classification_report
cv = CountVectorizer(max_features=30000,ngram_range=(1, 3))
pipeline = Pipeline([
        ('vectorizer', cv),
        ('classifier', clf)
    ])
sentiment_fit = pipeline.fit(X_train, y_train)
y_pred = sentiment_fit.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['negative','positive']))

              precision    recall  f1-score   support

    negative       0.75      0.63      0.68      5026
    positive       0.90      0.94      0.92     17496

    accuracy                           0.87     22522
   macro avg       0.82      0.78      0.80     22522
weighted avg       0.86      0.87      0.86     22522

